# 生成轨迹特征

1. 将每个用户的所有plt文件都合并为1个csv文件。
2. 生成停留点和移动点。
3. 删除异常值。
   1. 删除超过设定城市区域范围之外的轨迹。
   2. 停留时间超过一周的停留点。因为数据是都按天提供的。
4. 生成指定数据格式的轨迹数据。
   1. 生成时序数据。
   2. 矩阵数据。

In [1]:
import os
import pandas as pd
import transbigdata as tbd
from typing import Tuple, List
import random
import math
import numpy as np

from Utils import CalcGrid, OperJson

gParameters = OperJson.JSONConfig('./Parameters.json')
# print(gParameters.get('gPreprocessDataSavePath'))
# lon1, lat1, lon2, lat2
gGeoParameters = tbd.area_to_params(location = gParameters.get('gBoundsBeijing'), 
                                    accuracy = gParameters.get('gAccuracy'), 
                                    method = gParameters.get('gMethod'))

In [2]:
CG_Ellipsoid = CalcGrid.GridMapperEllipsoid(gParameters.get('gBoundsBeijing')[0], 
                         gParameters.get('gBoundsBeijing')[1], 
                         gParameters.get('gBoundsBeijing')[2],
                         gParameters.get('gBoundsBeijing')[3],
                         cell_size_m=1000)

testgrid = CG_Ellipsoid.lonlat_to_grid(117.52, 41.05)
testgrid

34203

In [ ]:
gUsersList = next(os.walk(gParameters.get('gTrajectoryFolderPath')))[1]
# gUsersList


def GenerateStayMove(userID:str) -> bool:
    """_summary_
    对一个用户生成停留点和移动特征。
    Args:
        userID (str): 用户的ID。

    Returns:
        bool: 是否报错，用于在上级循环中退出。
    """
    
    ErrorFlag = False
    gTrajectoryFolderPath = gParameters.get('gTrajectoryFolderPath')
    userdata = gTrajectoryFolderPath + '/{}/Trajectory/'.format(userID)

    # 返回指定路径下所有文件和文件夹的名字，并存放于一个列表中
    filelist = os.listdir(userdata)
    # plt文件中的字段名称。
    names = ['lat','lng','zero','alt','days','date','time']
    # 读取一个用户目录下的所有轨迹文件。
    df_list = [pd.read_csv(userdata + f, header=6, names=names, index_col=False) for f in filelist]
    # 将轨迹文件合并。
    df = pd.concat(df_list, ignore_index=True)
    # 规范时间格式。
    df['entireTime'] = pd.to_datetime((df['date'] + ' ' + df['time']), format='%Y-%m-%d %H:%M:%S')
    # 修改列名。
    df.rename(columns={'lat': 'latitude', 'lng': 'longitude'}, inplace=True)
    # 删除不需要使用的列。
    df.drop(['zero', 'days', 'date', 'time', 'alt'], axis=1, inplace=True)

    # print(df.shape)
    # 删除超过定义范围之外的点。
    df = tbd.clean_outofbounds(df, bounds = gParameters.get('gBoundsBeijing'), 
                            col = ['longitude', 'latitude'])
    # print(df.shape)
    if df.shape[0] == 0:
        print(f'{userID} after clean out of bounds, dataframe is null.')
        ErrorFlag = True
        return ErrorFlag
    # 添加userID列。
    df['userID'] = userID
    # df.head(3)

    # 生成停留点。
    stay, move = tbd.traj_stay_move(df, gGeoParameters,
                                        col=['userID', 'entireTime', 'longitude', 'latitude'], 
                                        activitytime=gParameters.get('gActivityTime'))
    if stay.shape[0] == 0:
        print(f'{userID} generate null stay, dataframe is null.')
        ErrorFlag = True
        return ErrorFlag
    
    # 删除停留点时间超过一周的停留点。
    # 因为geolife 中都是按天提供轨迹数据的。其中有部分数据的时间间隔是间断（超过一天）的。所以需要删除异常值。
    # print(stay)
    stay = stay[stay['duration'] <= gParameters.get('gStayDurationthreshold')]
    if stay.shape[0] == 0:
        print(f'{userID} Duration is less than the threshold, dataframe is null.')
        ErrorFlag = True
        return ErrorFlag

    # print(stay.columns)
    # 生成自定义的grid。
    def GenerateStayGrid(df):
        df['grid'] = CG_Ellipsoid.lonlat_to_grid(df['lon'], df['lat'])
        return df
    stay = stay.apply(GenerateStayGrid, axis=1)

    # 删除没有使用的列名。
    stay = stay[['userID', 'stime', 'etime', 'lon', 'lat', 'duration', 'grid']]
    stay.to_csv(gParameters.get('gSingleUserStaySavePath').format(userID))

    def GenerateMoveGrid(df):
        df['sgrid'] = CG_Ellipsoid.lonlat_to_grid(df['slon'], df['slat'])
        df['egrid'] = CG_Ellipsoid.lonlat_to_grid(df['elon'], df['elat'])
        return df
    move = move.apply(GenerateMoveGrid, axis=1)
    move = move[['userID', 'stime', 'slon', 'slat', 'sgrid', 'etime', 'elon', 'elat', 'egrid', 'duration']]
    # move
    move.to_csv(gParameters.get('gSingleUserMoveSavePath').format(userID))
    # print(f'{userID} has generated stay and move.')
    return ErrorFlag


# GenerateStayMove('049')

for userID in gUsersList:
    if GenerateStayMove(userID=userID):
        # raise ValueError(f"{userID} report ERROR.")
        continue

# consume 1m 38.4s .

049 generate null stay, dataframe is null.
118 after clean out of bounds, dataframe is null.
120 generate null stay, dataframe is null.
123 generate null stay, dataframe is null.
132 after clean out of bounds, dataframe is null.
137 generate null stay, dataframe is null.
160 after clean out of bounds, dataframe is null.
178 generate null stay, dataframe is null.
180 Duration is less than the threshold, dataframe is null.


generate record:

- 049 generate null stay, dataframe is null.
- 118 after clean out of bounds, dataframe is null.
- 120 generate null stay, dataframe is null.
- 123 generate null stay, dataframe is null.
- 132 after clean out of bounds, dataframe is null.
- 137 generate null stay, dataframe is null.
- 160 after clean out of bounds, dataframe is null.
- 178 generate null stay, dataframe is null.
- 180 Duration is less than the threshold, dataframe is null.

# 将每个人的轨迹都合并为一个CSV文件用于模型训练

In [ ]:
# 获取该路径下的所有文件。
filelist = os.listdir("./Data/Output/Stays/")
# 读取所有的CSV，并且存入链表。
df_list = [pd.read_csv("./Data/Output/Stays/" + f, index_col=0, dtype={'userID': object}) for f in filelist]
# 合并。
AllStays = pd.concat(df_list, ignore_index=True)
# 显示形状。
AllStays.shape

(27919, 7)

In [39]:
AllStays.head(3)

,userID,stime,etime,lon,lat,duration,grid
0,0,2008-10-23 03:01:40,2008-10-23 04:08:07,116.301309,39.984345,3987.0,14428
1,0,2008-10-23 04:30:57,2008-10-23 09:44:15,116.322261,39.998151,18798.0,14808
2,0,2008-10-23 09:44:15,2008-10-23 10:14:21,116.321396,40.007122,1806.0,14809


In [52]:
# 保存。
AllStays.to_csv('./Data/Output/AllUserTimeSeries.csv')

# 生成交互矩阵

In [51]:
InteractionMatrix = pd.pivot_table(AllStays[['userID', 'grid', 'duration']], index='userID',columns='grid', values='duration', aggfunc='count')
InteractionMatrix.fillna(0, inplace=True)
InteractionMatrix.to_csv('./Data/Output/InteractionMatrix.csv')

# 生成类似自然语言的Grid矩阵

1. 按8：2的比例分了训练集和测试集。
2. 可调参的矩阵第二个维度。

In [2]:
def list_split(input_list:List, ratio:float, shuffle=False)->Tuple[List, List]:
    """_summary_
    数据集拆分: 将列表full_list按比例ratio（随机）划分为2个子列表sublist_1与sublist_2
    Args:
        input_list (List): 数据列表
        ratio (float): 划分比例。
        shuffle (bool, optional): 是否打乱. Defaults to False.

    Returns:
        Tuple[List, List]: 返回划分好的两个链表。
    """
    full_list = input_list
    n_total = len(full_list)
    offset = int(n_total * ratio)
    if n_total == 0 or offset < 1:
        return [], full_list
    if shuffle:
        # 打乱。
        random.shuffle(full_list)
    # 截取之后排序。
    sublist_1 = sorted(full_list[:offset])
    sublist_2 = sorted(full_list[offset:])
    return sublist_1, sublist_2
 
# 测试。
data = list(range(20))
sub_data1, sub_data2 = list_split(data, ratio=0.2, shuffle=True)
print(sub_data1, sub_data2)


[4, 8, 13, 16] [0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 14, 15, 17, 18, 19]


In [3]:
# 生成训练集和测试集。
filelist = os.listdir("./Data/Output/Stays/")
trainUsers, testUsers = list_split(filelist, ratio=0.9, shuffle=True)
print(len(trainUsers), len(testUsers))

155 18


In [4]:
def FormatTrajData(UserFileName:List) -> Tuple[pd.DataFrame, List]:
    """_summary_
    将数据转化为类似于自然语言的矩阵形式。
    Args:
        UserFileName (List): 所有用户的csv文件名的list列表。

    Returns:
        Tuple[pd.DataFrame, List]: 返回将所有用户合并之后的数据集， 有些用户的停留点数量小于需要转换为矩阵的第二个维度，所以直接丢弃了，丢弃的记录下来的列表。
    """
    df_list = []
    delete_list = []
    for f in UserFileName:
        df = pd.read_csv("./Data/Output/Stays/" + f, index_col=0, dtype={'userID': object}) 
        if df.shape[0] < gParameters.get('gMatrixSecondDimension'):
            delete_list.append(f)
            continue
        # print(df.shape)
        if df.shape[0] % gParameters.get('gMatrixSecondDimension') != 0:
            # print(math.floor(df.shape[0]/gParameters.get('gMatrixSecondDimension')) * gParameters.get('gMatrixSecondDimension'))
            temp = df.iloc[:math.floor(df.shape[0]/gParameters.get('gMatrixSecondDimension')) * gParameters.get('gMatrixSecondDimension'), :]
            # print(temp.shape)
            df_list.append(temp)
    df = pd.concat(df_list, ignore_index=True)
    return df, delete_list


trainData, trainDelUsers = FormatTrajData(trainUsers)
testData, testDelUsers = FormatTrajData(testUsers)

In [5]:
def data_split_twodimension(sequence, windows_length=10, step_length=1):
    """_summary_
    将一个二维矩阵通过滑动窗口划分为样本和标签。
    标签取样本之后的第一个值。
    这个序列是按行进行的时间序列。
    Args:
        sequence (numpy.array): 三维矩阵，类似于文本的形状。
        windows_length (int, optional): 滑动窗口的长度. Defaults to 100.
        step_length (int, optional): 移动滑动窗口的步长. Defaults to 1.
    Returns:
        np.array(x) (numpy.array): 二维数组，样本。
        np.array(y) (numpy.array): 二维数组，标签。
    """
    x = []
    y = []
    
    for i in range(math.ceil(len(sequence)/step_length)):
        labelIndex = step_length * i + windows_length
        if labelIndex > len(sequence) - 1:
            break
        # sequence[i:labelIndex, :], sequence[labelIndex, :]
        seq_x, seq_y = sequence[step_length*i:labelIndex, :], sequence[labelIndex, :]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)

In [8]:
trainNp = trainData['grid'].values.reshape(-1, gParameters.get('gMatrixSecondDimension'))  # type: ignore
testNp = testData['grid'].values.reshape(-1, gParameters.get('gMatrixSecondDimension')) # type: ignore
print(trainNp.shape, testNp.shape)

trainSrc = trainNp[:-1, :]
trainTgt = trainNp[1:, :]

testSrc = testNp[:-1, :]
testTgt = testNp[1:, :]

print(trainSrc.shape, trainTgt.shape)

(2309, 10) (190, 10)
(2308, 10) (2308, 10)


In [9]:
np.savetxt('./Data/Output/StayTrainMatrix_{}.csv'.format(gParameters.get('gMatrixSecondDimension')), 
           trainNp, delimiter=',', fmt='%d')
np.savetxt('./Data/Output/StayTestMatrix_{}.csv'.format(gParameters.get('gMatrixSecondDimension')), 
           testNp, delimiter=',', fmt='%d')

# 生成提示词


参考：
1. <https://github.com/xiyuanzh/awesome-llm-time-series>
2. LLM recommendation<https://github.com/WLiK/LLM4Rec-Awesome-Papers>
3. 这里面包含一部分提示词<https://github.com/zyang1580/CoLLM/tree/main/prompts>。



```bash
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
    你是一名轨迹预测专家，理解时空序列与移动行为。  
    请根据以下历史轨迹（每个点包含时间、lon、lat、grid、duration），预测用户下一时刻的停留点经纬度与网格编号。

    <|eot_id|>

    <|start_header_id|>user<|end_header_id|>
    历史轨迹（按时间顺序，从最早到最近）：
    [
    {"stime":"2008-10-23 03:01:40","etime":"2008-10-23 04:08:07","lon":116.301309,"lat":39.984345,"duration":3987.0,"grid":14428},
    {"stime":"2008-10-23 04:30:57","etime":"2008-10-23 09:44:15","lon":116.322261,"lat":39.998151,"duration":18798.0,"grid":14808},
    {"stime":"2008-10-23 09:44:15","etime":"2008-10-23 10:14:21","lon":116.321396,"lat":40.007122,"duration":1806.0,"grid":14809}
    ]

    请预测下一个停留点，输出如下 JSON：

    {
    "next_stime": "...",
    "next_etime": "...",
    "pred_lon": ...,
    "pred_lat": ...,
    "pred_grid": ...
    }
    <|eot_id|>

    <|start_header_id|>assistant<|end_header_id|>
```

In [13]:
import pandas as pd
AllStays = pd.read_csv('./Data/Output/AllUserTimeSeries.csv', index_col=0)
AllStays.shape

(27919, 7)

In [14]:
AllStays.head(3)

,userID,stime,etime,lon,lat,duration,grid
0,0,2008-10-23 03:01:40,2008-10-23 04:08:07,116.301309,39.984345,3987.0,14428
1,0,2008-10-23 04:30:57,2008-10-23 09:44:15,116.322261,39.998151,18798.0,14808
2,0,2008-10-23 09:44:15,2008-10-23 10:14:21,116.321396,40.007122,1806.0,14809
